In [1]:
import os
import sys
import datetime
import numpy as np
import pandas as pd
import networkx as nx
#import dateparser

In [2]:
directory = "/home/cesar/Data/Dynamics/EmpiricalData/"

In [3]:
A = os.listdir(directory)
A.sort()
A

['Cit-HepTh-dates.txt',
 'Cit-HepTh.txt',
 'Temporal-Cit-HepTh.txt',
 'sx-mathoverflow.txt',
 'sx-stackoverflow.txt',
 'wiki-talk-temporal.txt']

In [4]:
#directory+'cit-HepPh-dates.txt'

In [5]:
#! less /home/cesar/Desktop/Doctorado/Projects/Networks/Embeddings/Data/cit-HepPh-dates.txt

In [6]:
#! less /home/cesar/Desktop/Doctorado/Projects/Networks/Embeddings/Data/Cit-HepPh.txt

In [15]:
def nodeReferenceToTemporal(directory,
                            temporalFileName,
                            networkFileName,
                            datesReferenceFileName):
    """
    Function for the Citation-Graphs
    """
    nodesDataFrame = pd.read_csv(directory+datesReferenceFileName,
                                 parse_dates=True,
                                 index_col=1,
                                 skiprows=0,
                                 sep="\t")
    
    A = nodesDataFrame.index.to_datetime()
    node2date = dict(zip(nodesDataFrame.values.transpose()[0],nodesDataFrame.index))
    maxDate = max(nodesDataFrame.index)
    minDate = min(nodesDataFrame.index)

    numberOfDaysInSeries = (maxDate - minDate).days
    edges = np.loadtxt(directory+networkFileName,skiprows=4)

    del(nodesDataFrame)
    del(A)
    errors = 0
    filebuffer = open(directory+temporalFileName,"w")
    edge_issues = []
    for edge in edges:
        NODEA = node_date(edge[0],node2date)
        NODEB = node_date(edge[1],node2date)
        try:
            edge_day = (max(NODEA,NODEB) - minDate).days
            edge_string = " ".join(map(str,map(int,np.concatenate([edge,[edge_day]]))))+"\n"
            filebuffer.write(edge_string)
        except:
            edge_issues.append((edge,sys.exc_info()))
            errors+=1
        if errors >= 100:
            break
            
    filebuffer.close()
    return edge_issues



In [4]:
def node_date(node,node2date):
    try:
        return node2date[int(edge[0])]
    except:
        try:
            return node2date[int("11"+str(edge[0]))]
        except:
            return None

In [5]:
%%capture
temporalFileName = 'Temporal-Cit-HepTh.txt'
networkFileName = 'Cit-HepTh.txt'
datesReferenceFileName = 'Cit-HepTh-dates.txt'

nodesDataFrame = pd.read_csv(directory+datesReferenceFileName,
                             parse_dates=True,
                             index_col=1,
                             skiprows=0,
                             sep="\t")

A = nodesDataFrame.index.to_datetime()
node2date = {}
for value, date in zip(nodesDataFrame.values.transpose()[0],nodesDataFrame.index):
    try:
        node2date[int(value)] = date
    except:
        pass
    
maxDate = max(nodesDataFrame.index)
minDate = min(nodesDataFrame.index)

numberOfDaysInSeries = (maxDate - minDate).days
edges = np.loadtxt(directory+networkFileName,skiprows=4)

del(nodesDataFrame)
del(A)

#edges_issues = nodeReferenceToTemporal(directory,temporalFileName,networkFileName,datesReferenceFileName)

In [8]:
errors = 0
filebuffer = open(directory+temporalFileName,"w")
edge_issues = []
for edge in edges:
    NODEA = node_date(edge[0],node2date)
    NODEB = node_date(edge[1],node2date)
    try:
        edge_day = (max(NODEA,NODEB) - minDate).days
        edge_string = " ".join(map(str,map(int,np.concatenate([edge,[edge_day]]))))+"\n"
        filebuffer.write(edge_string)
    except:
        edge_issues.append((edge,sys.exc_info()))
        #errors+=1
        pass
    if errors > 100:
        break
filebuffer.close()

In [138]:
gd_folder = "/home/cesar/Desktop/Doctorado/Projects/Networks/Dynamics/Simulations/cit-HepPh_gd/"
dynamical_identifier = "cit-HepPh"
temporalFileName

'Temporal-Cit-HepPh.txt'

In [139]:
def TemporalToSeries(gd_folder,dynamics_identifier,temporalFileName,stepsInGraph=7):
    """
    """
    temporal_edges = np.loadtxt(temporalFileName,delimiter=" ")
    days = temporal_edges[:,2]
    minDay = int(min(days))
    maxDay = int(max(days))
    for time_index, current_day in enumerate(range(minDay,maxDay,stepsInGraph)):
        graph_file_name  = gd_folder+"{0}_gGD_{1}_.gd".format(dynamics_identifier,time_index)
        current_edges = np.take(temporal_edges,np.where(days < current_day)[0],axis=0)[:,[0,1]]
        np.savetxt(graph_file_name,np.float_(current_edges))

In [140]:
TemporalToSeries(gd_folder,dynamical_identifier,directory+temporalFileName,)

In [141]:
(a,b,c) = (1,2,3)